In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import os
import holidays
import datetime as dt
import networkx as nx
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import glob
import networkx as nx

pd.set_option('display.max_columns', 70)
os.getcwd()

'/Users/miek/Documents/DS4B/DS4BThesisSNCB'

In [4]:
commercial_stops_example = pd.read_csv("./data/commercial_stops_example_trains_locseclocnext.csv")
commercial_stops_example

,REF_TOD_TRAIN_ID,TRAIN_NUMBER,TRAIN_DEPARTURE_DATE,REF_PTCAR_NO,PTCAR_ORD_NO,THR_ARR,THR_DEP,HR_ARR,HR_DEP,EH_ARR,EH_DEP,COMMERCIAL_STOP_NATURE,DEPARTURE_REF_LINE_SECTION_NO,LINE,IDENTIFICATOR,YEAR_DEP,month,season,weekend,holiday,dep_hour_origin,day_origin,date_origin,start_lifetime,end_lifetime,arr_hour_terminus,p_indicator,LONG_NAME_NL,GPS_LATITUDE,GPS_LONGITUDE,REF_PTCAR_NO_next,THR_ARR_next,LONG_NAME_NL_next,loc_sec_locnext
0,30282563,19951,2022-05-12,737,1,1900-01-01 00:00:00,2022-05-12 17:08:00,1900-01-01 00:00:00,2022-05-12 17:08:06,0,6,1,0,NaN,NaN,2022,5.0,Spring,0.0,0.0,17.133333,12.0,2022-05-12,2022-05-12 17:08:00,2022-05-12 17:36:00,17.600000,0,NaN,NaN,NaN,73.0,1900-01-01 17:17:00,NaN,737_0_73.0
1,30286599,4715,2022-05-12,769,1,1900-01-01 00:00:00,2022-05-12 15:20:00,1900-01-01 00:00:00,2022-05-12 15:20:46,0,46,1,0,NaN,NaN,2022,5.0,Spring,0.0,0.0,15.333333,12.0,2022-05-12,2022-05-12 15:20:00,2022-05-12 15:49:00,15.816667,0,NaN,NaN,NaN,1162.0,1900-01-01 15:23:00,NaN,769_0_1162.0
2,30287378,9855,2022-05-12,802,1,1900-01-01 00:00:00,2022-05-12 06:36:00,1900-01-01 00:00:00,2022-05-12 06:35:45,0,-15,1,0,NaN,NaN,2022,5.0,Spring,0.0,0.0,6.600000,12.0,2022-05-12,2022-05-12 06:36:00,2022-05-12 12:11:00,12.183333,0,NaN,NaN,NaN,1936.0,1900-01-01 06:48:00,NaN,802_0_1936.0
3,30284788,9240,2022-05-12,88,1,1900-01-01 00:00:00,2022-05-12 13:28:00,1900-01-01 00:00:00,2022-05-12 13:28:11,0,11,1,0,NaN,NaN,2022,5.0,Spring,0.0,0.0,13.466667,12.0,2022-05-12,2022-05-12 13:28:00,2022-05-12 16:17:00,16.283333,0,NaN,NaN,NaN,1064.0,1900-01-01 13:41:00,NaN,88_0_1064.0
4,30283474,4709,2022-05-12,769,1,1900-01-01 00:00:00,2022-05-12 09:20:00,1900-01-01 00:00:00,2022-05-12 09:21:33,0,93,1,0,NaN,NaN,2022,5.0,Spring,0.0,0.0,9.333333,12.0,2022-05-12,2022-05-12 09:20:00,2022-05-12 09:49:00,9.816667,0,NaN,NaN,NaN,1162.0,1900-01-01 09:23:00,NaN,769_0_1162.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53020,30594260,18454,2022-07-02,723,194,2022-07-02 21:53:00,2022-07-02 21:56:00,2022-07-02 21:49:26,2022-07-02 21:56:10,-214,10,2,1053,162.0,162,2022,7.0,Summer,1.0,0.0,18.083333,2.0,2022-07-02,2022-07-02 18:05:00,2022-07-02 22:22:00,22.366667,0,LIBRAMONT,49.920425,5.379272,786.0,1900-01-01 22:09:00,MARBEHAN,LIBRAMONT_1053_MARBEHAN
53021,30594205,18453,2022-07-02,723,194,2022-07-02 20:53:00,2022-07-02 20:56:00,2022-07-02 20:49:31,2022-07-02 20:56:08,-208,8,2,1053,162.0,162,2022,7.0,Summer,1.0,0.0,17.083333,2.0,2022-07-02,2022-07-02 17:05:00,2022-07-02 21:22:00,21.366667,0,LIBRAMONT,49.920425,5.379272,786.0,1900-01-01 21:09:00,MARBEHAN,LIBRAMONT_1053_MARBEHAN
53022,30594260,18454,2022-07-02,786,197,2022-07-02 22:09:00,2022-07-02 22:10:00,2022-07-02 22:08:49,2022-07-02 22:10:03,-10,3,2,1056,162.0,162,2022,7.0,Summer,1.0,0.0,18.083333,2.0,2022-07-02,2022-07-02 18:05:00,2022-07-02 22:22:00,22.366667,0,MARBEHAN,49.727421,5.539753,70.0,1900-01-01 22:22:00,ARLON,MARBEHAN_1056_ARLON
53023,30594205,18453,2022-07-02,786,197,2022-07-02 21:09:00,2022-07-02 21:10:00,2022-07-02 21:09:12,2022-07-02 21:10:57,12,57,2,1056,162.0,162,2022,7.0,Summer,1.0,0.0,17.083333,2.0,2022-07-02,2022-07-02 17:05:00,2022-07-02 21:22:00,21.366667,0,MARBEHAN,49.727421,5.539753,70.0,1900-01-01 21:22:00,ARLON,MARBEHAN_1056_ARLON


In [8]:
commercial_stops_example['THR_ARR_next'] = pd.to_datetime(commercial_stops_example['THR_ARR_next'], format='%Y-%m-%d %H:%M:%S')
commercial_stops_example['THR_ARR_next_3min'] = commercial_stops_example['THR_ARR_next'] + timedelta(minutes=3)
commercial_stops_example = commercial_stops_example[['REF_TOD_TRAIN_ID','TRAIN_NUMBER','THR_ARR_next','loc_sec_locnext','THR_ARR_next_3min']]
commercial_stops_example.head(5)

,REF_TOD_TRAIN_ID,TRAIN_NUMBER,THR_ARR_next,loc_sec_locnext,THR_ARR_next_3min
0,30282563,19951,1900-01-01 17:17:00,737_0_73.0,1900-01-01 17:20:00
1,30286599,4715,1900-01-01 15:23:00,769_0_1162.0,1900-01-01 15:26:00
2,30287378,9855,1900-01-01 06:48:00,802_0_1936.0,1900-01-01 06:51:00
3,30284788,9240,1900-01-01 13:41:00,88_0_1064.0,1900-01-01 13:44:00
4,30283474,4709,1900-01-01 09:23:00,769_0_1162.0,1900-01-01 09:26:00


In [9]:
commercial_stops_example.dtypes

REF_TOD_TRAIN_ID              int64
TRAIN_NUMBER                  int64
THR_ARR_next         datetime64[ns]
loc_sec_locnext              object
THR_ARR_next_3min    datetime64[ns]
dtype: object

In [22]:
folder_path = '/Users/miek/Downloads/adj_matrices_comstops_example'
commercial_stops_example = commercial_stops_example.sort_values(by = 'TRAIN_NUMBER')

grouped_data = []

grouped = commercial_stops_example.groupby(['loc_sec_locnext'])
for name, group in grouped:
    group = group.drop_duplicates(subset=['TRAIN_NUMBER','loc_sec_locnext'],keep = 'last')
    ref_ids = group['TRAIN_NUMBER'].values
    mask = (ref_ids[:, np.newaxis] != ref_ids) & \
    (group['THR_ARR_next'].values[:, np.newaxis] >= group['THR_ARR_next'].values) & \
    (group['THR_ARR_next'].values[:, np.newaxis] <= group['THR_ARR_next_3min'].values)
    ohe = pd.get_dummies(group['TRAIN_NUMBER'])
    np.fill_diagonal(ohe.values, 0)
    group_ohe = pd.concat([group, ohe], axis=1)
    column_index = group_ohe.columns.get_loc('THR_ARR_next_3min') + 1
    group_ohe.iloc[:, column_index:] = np.where(mask, 1, group_ohe.iloc[:, column_index:])
    group_ohe = group_ohe.drop(columns = ['REF_TOD_TRAIN_ID','THR_ARR_next','THR_ARR_next_3min','loc_sec_locnext'])
    group_ohe = group_ohe.set_index(['TRAIN_NUMBER'])
    group_ohe = group_ohe.T
    group_ohe['loc_sec_locnext'] = name[0]
    name_with_underscores = name[0].replace(" ", "-").replace('/', '').replace(':', '')
    filename = f"{name_with_underscores}.csv"  # Using 'LONG_NAME_NL' and day in the filename
    #Save the group dataframe as a CSV file in the specified folder
    group_ohe.to_csv(os.path.join(folder_path, filename), index=True)


In [23]:
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
csv_files.sort()

dataframes = [pd.read_csv(os.path.join(folder_path, file)) for file in csv_files]
concatenated_df = pd.concat(dataframes, ignore_index=True)
concatenated_df_copy = concatenated_df.fillna(0)
concatenated_df_copy

,Unnamed: 0,2577,2578,2579,2580,2581,2582,2583,2584,2585,2586,2587,2588,2589,2590,2591,2592,2593,2594,19618,19693,loc_sec_locnext,9212,9216,9220,9224,9228,9232,9236,9240,9244,9248,9252,9256,9260,...,19910,19912,19914,19916,19918,19920,19922,19924,19928,19930,19932,19934,19938,19940,19942,19944,19946,19948,19952,19954,19956,19958,19960,19962,19964,19966,19968,19970,19972,19974,19976,17242,5005,9492,503
0,2577,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1015_0_ESSEN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1015_0_ESSEN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1015_0_ESSEN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2580,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1015_0_ESSEN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1015_0_ESSEN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53020,2669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ZWIJNDRECHT_1312_MELSELE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53021,2670,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ZWIJNDRECHT_1312_MELSELE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53022,2671,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ZWIJNDRECHT_1312_MELSELE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53023,2672,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ZWIJNDRECHT_1312_MELSELE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
concatenated_df_copy = concatenated_df_copy.rename(columns={'Unnamed: 0': 'TRAIN_NUMBER'})
concatenated_df_copy

,TRAIN_NUMBER,2577,2578,2579,2580,2581,2582,2583,2584,2585,2586,2587,2588,2589,2590,2591,2592,2593,2594,19618,19693,loc_sec_locnext,9212,9216,9220,9224,9228,9232,9236,9240,9244,9248,9252,9256,9260,...,19910,19912,19914,19916,19918,19920,19922,19924,19928,19930,19932,19934,19938,19940,19942,19944,19946,19948,19952,19954,19956,19958,19960,19962,19964,19966,19968,19970,19972,19974,19976,17242,5005,9492,503
0,2577,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1015_0_ESSEN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1015_0_ESSEN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1015_0_ESSEN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2580,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1015_0_ESSEN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1015_0_ESSEN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53020,2669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ZWIJNDRECHT_1312_MELSELE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53021,2670,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ZWIJNDRECHT_1312_MELSELE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53022,2671,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ZWIJNDRECHT_1312_MELSELE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53023,2672,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ZWIJNDRECHT_1312_MELSELE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
concatenated_df_copy = concatenated_df_copy.groupby('TRAIN_NUMBER').sum().reset_index()
concatenated_df_copy 

,TRAIN_NUMBER,2577,2578,2579,2580,2581,2582,2583,2584,2585,2586,2587,2588,2589,2590,2591,2592,2593,2594,19618,19693,loc_sec_locnext,9212,9216,9220,9224,9228,9232,9236,9240,9244,9248,9252,9256,9260,...,19910,19912,19914,19916,19918,19920,19922,19924,19928,19930,19932,19934,19938,19940,19942,19944,19946,19948,19952,19954,19956,19958,19960,19962,19964,19966,19968,19970,19972,19974,19976,17242,5005,9492,503
0,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1_0_LIEGE-GUILLEMINS659_0_1.0BRUSSEL-NOORD_143...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BRUSSEL-NOORD_151_LIEGE-GUILLEMINSBRUSSEL-ZUID...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1_0_LIEGE-GUILLEMINS659_0_1.0BRUSSEL-NOORD_143...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BRUSSEL-NOORD_151_LIEGE-GUILLEMINSBRUSSEL-ZUID...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1_0_LIEGE-GUILLEMINS659_0_1.0BRUSSEL-NOORD_143...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5693,19973,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,737_0_73.073_0_FROYENNESFROYENNES_395_TOURNAI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5694,19974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOURNAI_395_FROYENNES,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5695,19975,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,737_0_73.073_0_FROYENNESFROYENNES_395_TOURNAI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5696,19976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOURNAI_395_FROYENNES,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
concatenated_df_copy.dtypes

TRAIN_NUMBER      int64
2577            float64
2578            float64
2579            float64
2580            float64
                 ...   
19976           float64
17242           float64
5005            float64
9492            float64
503             float64
Length: 5700, dtype: object

In [31]:
unique_elements, counts = np.unique(concatenated_df_copy.drop(columns = ['loc_sec_locnext', 'TRAIN_NUMBER']), return_counts=True)
value_counts = dict(zip(unique_elements, counts))
value_counts

{0.0: 32464423,
 1.0: 913,
 2.0: 595,
 3.0: 282,
 4.0: 286,
 5.0: 208,
 6.0: 141,
 7.0: 84,
 8.0: 82,
 9.0: 93,
 10.0: 22,
 11.0: 58,
 12.0: 2,
 13.0: 1,
 16.0: 4,
 17.0: 10}